# NIH Chest X-ray Dataset Explanation

## Introduction
The **NIH Chest X-ray Dataset** is a large-scale dataset provided by the National Institutes of Health (NIH), containing **112,120 frontal-view chest X-ray images** from **30,805 unique patients**. The dataset was introduced to support the development of deep learning models for automated diagnosis of thoracic diseases.

## Key Features
- **Large-scale dataset**: Contains over 112,000 X-ray images.
- **Multi-label classification**: Includes annotations for **14 different thoracic diseases**, such as pneumonia, edema, emphysema, and fibrosis.
- **Metadata Availability**: Includes patient age, gender, view position, and image index.
- **Open-Source**: Publicly available for research and development in medical AI applications.

## Labels and Diseases
The dataset provides labels for the following 14 thoracic conditions:
- Atelectasis
- Cardiomegaly
- Consolidation
- Edema
- Effusion
- Emphysema
- Fibrosis
- Hernia
- Infiltration
- Mass
- Nodule
- Pleural Thickening
- Pneumonia
- Pneumothorax

```

## Applications
- **Automated disease diagnosis** using deep learning models.
- **Medical AI research** for improving diagnostic accuracy.
- **Explainability and interpretability studies** to analyze model decision-making.

The NIH Chest X-ray Dataset has been widely used in medical imaging research, contributing to advancements in AI-powered diagnostics.

"""


# DenseNet201 Model Explanation

## Introduction
DenseNet201 is a deep convolutional neural network that belongs to the DenseNet family, which stands for **Densely Connected Convolutional Networks**. It was introduced in the paper *"Densely Connected Convolutional Networks"* by Gao Huang et al. in 2017.

## Key Features
- **Dense Connectivity**: Each layer receives input from all preceding layers and passes its feature maps to all subsequent layers.
- **Efficient Parameter Utilization**: Due to the dense connections, the network requires fewer parameters compared to traditional architectures.
- **Alleviation of Vanishing Gradient Problem**: By allowing shorter connections between layers close to the input and those close to the output, DenseNet helps in better gradient flow.
- **Feature Reuse**: Enhances learning efficiency by enabling the reuse of previously learned features.

## Architecture
DenseNet201 consists of:
- **Convolutional Layer**: Initial feature extraction.
- **Dense Blocks**: Several layers with Batch Normalization, ReLU, and Convolution, densely connected.
- **Transition Layers**: Reduces dimensionality using 1x1 convolution and pooling layers.
- **Global Average Pooling**: Reduces spatial dimensions before fully connected layers.
- **Fully Connected Layer**: Final classification output.

## Model Usage in TensorFlow/Keras
```python
from tensorflow.keras.applications import DenseNet201

# Load pre-trained DenseNet201 model
model = DenseNet201(weights='imagenet', include_top=True)
model.summary()
```

DenseNet201 is widely used in image classification tasks and can be fine-tuned for various applications, including medical imaging and object detection.

"""


## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# ML tools 
import tensorflow as tf
# from kaggle_datasets import KaggleDatasets
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from tensorflow.keras import layers, Model, optimizers
# import tensorflow.keras.applications.efficientnet as efn
from tensorflow.keras.applications import *
import os
# from tensorflow.keras.applications import DenseNet201 

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

## Data Load and preprocessing

In [ ]:
df = pd.read_csv('./Data_Entry_2017.csv')
df.img_ind= df.img_ind.apply(lambda x: x.split('.')[0])


# df["img_ind"] = "NIH_Images\\" + df["img_ind"]

# NIH_Images\00012255_000.jpg

display(df.head(4))
print(df.shape)

In [ ]:
directory = "./NIH_images"  # Change to your target directory
files = os.listdir(directory)  # List all files in the directory
files = sorted(files)  # Optional: Sort files alphabetically

print("Top 5 files:", files[:5])  # Print first 5 files


In [ ]:
target_cols = df.drop(['img_ind'], axis=1).columns.to_list()
n_classes = len(target_cols)
img_size = 600
n_epochs = 35
lr= 0.0001
seed= 11
val_split= 0.2
seed= 33
batch_size=12
n_classes

## Data preparation for faster ingestion during training

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

'''
Reference
https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training

'''

def build_decoder(with_labels=True, target_size=(img_size, img_size), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path) # Reads and outputs the entire contents of the input filename.

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3) # Decode a PNG-encoded image to a uint8 or uint16 tensor
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3) # Decode a JPEG-encoded image to a uint8 tensor
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0 # Casts a tensor to the type float32 and divides by 255.
        img = tf.image.resize(img, target_size) # Resizing to target size
        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO) # overlaps data preprocessing and model execution while training
    return dset


In [ ]:
strategy = auto_select_accelerator()
batch_size = strategy.num_replicas_in_sync * batch_size
print('batch size', batch_size)

In [ ]:
paths = "./NIH_images/" + df['img_ind'] + '.jpg'

print(paths)
#Get the multi-labels
label_cols = df.columns[:-1]
labels = df[label_cols].values

## Train test split

In [ ]:
(train_paths, valid_paths, 
  train_labels, valid_labels) = train_test_split(paths, labels, test_size=val_split, random_state=11)

print(train_paths.shape, valid_paths.shape)
train_labels.sum(axis=0), valid_labels.sum(axis=0)

In [ ]:
# Build the tensorflow datasets

decoder = build_decoder(with_labels=True, target_size=(img_size, img_size))

# Build the tensorflow datasets
dtrain = build_dataset(
    train_paths, train_labels, bsize=batch_size, decode_fn=decoder
)

dvalid = build_dataset(
    valid_paths, valid_labels, bsize=batch_size, 
    repeat=False, shuffle=False, augment=False, decode_fn=decoder
)

In [ ]:
data, _ = dtrain.take(2)
images = data[0].numpy()

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(20,10))
axes = axes.flatten()
for img, ax in zip(images, axes):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.show()

## Model Building

In [ ]:
# Define the Model Using DenseNet201
def build_model():
    base = DenseNet201(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')

    inp = layers.Input(shape=(img_size, img_size, 3))
    x = base(inp)
    x = layers.GlobalAveragePooling2D()(layers.Dropout(0.16)(x))
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(n_classes, activation='sigmoid')(x)  # Multi-label classification

    return Model(inp, x)

    

In [ ]:
with strategy.scope():
    model= build_model()
    loss= tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0)
    model.compile(optimizers.Adam(learning_rate=lr),loss=loss,metrics=[tf.keras.metrics.AUC(multi_label=True)])


In [ ]:
model.summary()

In [ ]:
# name= 'NIH_Seresnet152_model.h5'

# rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 2, verbose = 1, 
#                                 min_delta = 1e-4, min_lr = 1e-6, mode = 'min', cooldown=1)
        
# ckp = ModelCheckpoint(name,monitor = 'val_loss',
#                       verbose = 1, save_best_only = True, mode = 'min')
        
# es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 5, mode = 'min', 
#                     restore_best_weights = True, verbose = 1)

name = "NIH_DenseNet201_model.h5"

rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, 
                        min_delta=1e-4, min_lr=1e-6, mode='min', cooldown=1)

ckp = ModelCheckpoint(name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, mode='min', 
                   restore_best_weights=True, verbose=1)

In [ ]:
steps_per_epoch = (train_paths.shape[0] // batch_size)
steps_per_epoch

In [ ]:
history = model.fit(dtrain,                      
                    validation_data=dvalid,                                       
                    epochs=n_epochs,
                    callbacks=[rlr,es,ckp],
                    steps_per_epoch=steps_per_epoch,
                    verbose=1)

In [ ]:
"workspace/NIH_DenseNet201_model.h5"

model.save('./NIH_DenseNet201_model.keras')

## Model Evaluation

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot( history.history["loss"], label = "Training Loss", marker='o')
plt.plot( history.history["val_loss"], label = "Validation Loss", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.plot( history.history["auc"], label = "Training AUC" , marker='o')
plt.plot( history.history["val_auc"], label = "Validation AUC", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
tf.keras.backend.clear_session()

from sklearn.metrics import roc_auc_score

model= tf.keras.models.load_model(name,  safe_mode=False)
pred= model.predict(dvalid, verbose=1)

print('AUC CKECK-UP per CLASS')

classes= df.columns[:-1]
for i, n in enumerate(classes):
  print(classes[i])
  print(i, roc_auc_score(valid_labels[:, i], pred[:, i]))
  print('---------')

In [ ]:
# Clear session to free memory
tf.keras.backend.clear_session()

# Load the trained model
model = tf.keras.models.load_model(name, safe_mode=False)

# Get model predictions
pred = model.predict(dvalid, verbose=1)

# Define disease classes
classes = df.columns[:-1]

# Compute AUC scores for each class
auc_scores = []
for i, n in enumerate(classes):
    auc = roc_auc_score(valid_labels[:, i], pred[:, i])
    auc_scores.append(auc)

# Convert to NumPy array for easy sorting (optional)
auc_scores = np.array(auc_scores)

# **Plot the AUC scores**
plt.figure(figsize=(12, 6))
plt.barh(classes, auc_scores, color='royalblue')
plt.xlabel("AUC Score")
plt.ylabel("Disease Classes")
plt.title("AUC Score per Disease Class")
plt.xlim(0, 1)  # AUC scores range from 0 to 1
plt.grid(axis="x", linestyle="--", alpha=0.5)

# Annotate bars with values
for index, value in enumerate(auc_scores):
    plt.text(value + 0.02, index, f"{value:.3f}", va="center", fontsize=10)

# Show the plot
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

# Clear TensorFlow session
tf.keras.backend.clear_session()

# Load the trained model
model = tf.keras.models.load_model(name, safe_mode=False)

# Get model predictions
pred = model.predict(dvalid, verbose=1)

# Define disease classes
classes = df.columns[:-1]

# Initialize figure for multiple ROC curves
plt.figure(figsize=(10, 8))

# Loop through each disease class and plot its ROC curve
for i, disease in enumerate(classes):
    fpr, tpr, _ = roc_curve(valid_labels[:, i], pred[:, i])  # Compute FPR, TPR
    roc_auc = auc(fpr, tpr)  # Compute AUC
    plt.plot(fpr, tpr, label=f"{disease} (AUC = {roc_auc:.3f})")  # Plot each ROC curve

# Plot diagonal line (random classifier)
plt.plot([0, 1], [0, 1], 'k--', label="Random (AUC = 0.5)")

# Set axis labels and title
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curves for All Disease Classes")
plt.legend(loc="lower right")  # Add legend
plt.grid(alpha=0.3)  # Light grid for better readability

# Show the plot
plt.show()


## Model Explainability: Gradcam visualization

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

def create_gradcam_model(model):
    """
    Creates a model for Grad-CAM, identifying the base feature extractor 
    and the last convolutional layer.
    """
    # Print model summary to understand its structure
    print("Model structure:")
    model.summary(line_length=100)
    
    # First, find the feature extractor (DenseNet) and target dense layer
    feature_extractor = None
    dense_layer = None
    
    for layer in model.layers:
        if 'densenet' in layer.name:
            feature_extractor = layer
        if isinstance(layer, tf.keras.layers.Dense) and layer.name == 'dense':
            dense_layer = layer
    
    if feature_extractor is None:
        raise ValueError("Could not find DenseNet feature extractor in the model")
    if dense_layer is None:
        raise ValueError("Could not find final dense layer in the model")
    
    # Now, create a direct connection between input and feature extractor's output
    # This is a cleaner approach than trying to search for specific internal layers
    
    # Get the input
    model_input = model.input
    
    # Get the output of the feature extractor (before global pooling)
    # For DenseNet, this would be the feature maps before global pooling
    feature_maps = feature_extractor.output
    
    # Get the model's output (classification)
    predictions = model.output
    
    # Create a model that outputs both the feature maps and the predictions
    gradcam_model = Model(inputs=model_input, outputs=[feature_maps, predictions])
    
    return gradcam_model

def generate_gradcam(model, img_array, class_idx):
    """
    Generates a Grad-CAM heatmap using a direct approach.
    """
    # Create the Grad-CAM model
    gradcam_model = create_gradcam_model(model)
    
    # Use gradient tape to compute gradients
    with tf.GradientTape() as tape:
        # Cast the image to float32 to ensure proper gradient computation
        img_array = tf.cast(img_array, tf.float32)
        
        # Watch the input image
        tape.watch(img_array)
        
        # Get feature maps and predictions
        feature_maps, predictions = gradcam_model(img_array)
        
        # Get the prediction for the target class
        target_class_prediction = predictions[:, class_idx]
    
    # Calculate gradients of the target class prediction with respect to feature maps
    grads = tape.gradient(target_class_prediction, feature_maps)
    
    # Global average pooling of gradients
    pooled_grads = tf.reduce_mean(grads, axis=(1, 2))
    
    # Multiply feature maps with weighted gradients
    # This highlights the regions that contribute most to the target class
    feature_maps = feature_maps[0]
    pooled_grads = pooled_grads[0]
    
    # Apply gradient weights to each channel of the feature map
    heatmap = tf.zeros_like(feature_maps[:, :, 0])
    
    for i in range(pooled_grads.shape[0]):
        heatmap += feature_maps[:, :, i] * pooled_grads[i]
    
    # Apply ReLU to focus on positive contributions
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    # Convert to numpy array
    heatmap = heatmap.numpy()
    
    return heatmap

def overlay_gradcam(img_path, heatmap, alpha=0.5):
    """
    Overlays Grad-CAM heatmap on the original image.
    Args:
    - img_path: Path to the original image
    - heatmap: Grad-CAM heatmap
    - alpha: Transparency factor
    Returns:
    - superimposed_img: Image with heatmap overlay
    """
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize heatmap to match original image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Convert heatmap to RGB format
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    # Superimpose heatmap on the original image
    superimposed_img = cv2.addWeighted(heatmap, alpha, img, 1 - alpha, 0)
    
    return superimposed_img

# Main execution
# 🔹 Load your trained model
model = tf.keras.models.load_model(name)

# 🔹 Select the image and class to analyze
img_path = "./NIH_images/00030786_007.jpg"
img_size = 600  # Adjust based on your model's input size
class_idx = 2  # Example: Cardiomegaly

# 🔹 Preprocess the image
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_size, img_size))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize

# 🔹 Generate Grad-CAM heatmap
try:
    heatmap = generate_gradcam(model, img_array, class_idx)
    
    # 🔹 Overlay Grad-CAM on the original image
    superimposed_img = overlay_gradcam(img_path, heatmap)
    
    # 🔹 Plot the results
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img)
    ax[0].set_title("Original X-ray")
    ax[0].axis("off")
    ax[1].imshow(superimposed_img)
    ax[1].set_title("Grad-CAM Heatmap")
    ax[1].axis("off")
    plt.show()
except Exception as e:
    print(f"Error: {str(e)}")
    
    # Fallback to an even simpler approach if needed
    print("Attempting fallback approach...")
    
    # This is a last resort approach if everything else fails
    # It uses a simpler method that doesn't rely on internal model structure
    
    def simple_gradcam(model, img_array, class_idx):
        """
        A simplified Grad-CAM implementation that works with any model structure.
        """
        # Create a model that outputs both the final dense layer's input and the prediction
        # This assumes the model's last layers are GlobalAveragePooling followed by Dense
        for i, layer in enumerate(model.layers):
            if isinstance(layer, tf.keras.layers.Dense) and layer.name == 'dense':
                # Find the layer that feeds into this dense layer
                prev_layer = model.layers[i-1]
                break
        else:
            raise ValueError("Could not find the dense layer")
        
        # Create a model that gets the output of the layer before dense
        intermediate_model = Model(inputs=model.input, outputs=prev_layer.output)
        
        # Get the feature map representation
        features = intermediate_model.predict(img_array)
        
        # Get the prediction for the target class
        prediction = model.predict(img_array)[0, class_idx]
        
        # Create a gradient-weighted class activation map
        # Since we can't trace gradients easily this way, we'll use the feature importance
        # This is not a true Grad-CAM but a simplified visualization
        
        # Reshape the features to a 2D array and create a basic heatmap
        if len(features.shape) > 2:  # If features still have spatial dimensions
            heatmap = np.mean(features[0], axis=-1)
        else:
            # For global pooled features, we create a placeholder heatmap
            # This is just for visualization, not a true Grad-CAM
            heatmap = np.ones((10, 10))  # Placeholder
            
        return heatmap
    
    # Try the fallback approach
    heatmap = simple_gradcam(model, img_array, class_idx)
    superimposed_img = overlay_gradcam(img_path, heatmap)
    
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img)
    ax[0].set_title("Original X-ray")
    ax[0].axis("off")
    ax[1].imshow(superimposed_img)
    ax[1].set_title("Feature Visualization (Fallback)")
    ax[1].axis("off")
    plt.show()